# Análise Causal: O Impacto da Escola em Tempo Integral na Renda e Emprego

**Autor:** Seu Nome
**Data:** 15 de julho de 2025

## 1. Entendimento do Negócio e da Questão Causal

Este notebook implementa uma análise de inferência causal para estimar o impacto da expansão das Escolas de Tempo Integral (ETI) no Brasil sobre os resultados dos jovens no mercado de trabalho.

**Objetivo:** Ir além da correlação e estimar o **efeito causal** de frequentar uma ETI na probabilidade de emprego formal e na renda de jovens adultos.

**Metodologia:** Utilizaremos um modelo de **Diferenças em Diferenças (DiD)**, aplicando-o a um painel de dados construído a partir dos microdados da PNAD Contínua. Esta abordagem nos permite simular um experimento, comparando a trajetória de resultados de um "grupo de tratamento" (exposto à política) com um "grupo de controle".

**Estrutura CRISP-DM:**
1.  **Entendimento do Negócio:** Definir a questão causal.
2.  **Entendimento dos Dados:** Explorar a PNAD Contínua.
3.  **Preparação dos Dados:** Limpar, transformar e construir o painel.
4.  **Modelagem:** Estimar o modelo DiD e validar suas premissas.
5.  **Avaliação:** Interpretar os resultados e suas limitações.
6.  **Implantação:** Preparar os dados para um dashboard em Streamlit.

In [ ]:
# Instalação das bibliotecas necessárias para o projeto.
# - pandas: para manipulação e análise de dados.
# - statsmodels: para modelos estatísticos, incluindo regressão com pesos e erros clusterizados.
# - plotnine: para visualizações de dados elegantes baseadas na "grammar of graphics".
# - basedosdados: para facilitar o acesso a dados públicos brasileiros (alternativa ao download manual).

print("Instalando bibliotecas...")
!pip install pandas statsmodels plotnine basedosdados -q
print("Instalação concluída.")

Instalando bibliotecas...
Instalação concluída.


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from plotnine import ggplot, aes, geom_line, geom_vline, labs, theme_minimal, scale_color_manual, geom_ribbon
import basedosdados as bd
import warnings

# Configurações de visualização e avisos
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Bibliotecas importadas com sucesso.")

Bibliotecas importadas com sucesso.


In [ ]:
# Substitua pelo ID do seu projeto no Google Cloud Platform
BILLING_PROJECT_ID = "gen-lang-client-0926041388"

# Query para selecionar as variáveis de interesse da PNAD Contínua de 2016 a 2023 (4º trimestre de cada ano)
query = """
SELECT
    ano,
    sigla_uf,
    V2009 AS idade,
    V2007 AS sexo,
    V2010 AS cor_raca,
    V1028 AS peso_amostral,
    VD4002 AS condicao_ocupacao,
    VD4035 AS rendimento_trabalho_habitual
FROM
    `basedosdados.br_ibge_pnadc.microdados`
WHERE
    ano BETWEEN 2016 AND 2023
    AND trimestre = 4
    AND V2009 BETWEEN 20 AND 25
"""

print("Carregando dados da PNADc... Isso pode levar alguns minutos.")
df_pnad = bd.read_sql(query, billing_project_id=BILLING_PROJECT_ID)
print("Dados carregados com sucesso!")
print(f"Dimensões do DataFrame: {df_pnad.shape}")
df_pnad.head(100)

Carregando dados da PNADc... Isso pode levar alguns minutos.
Downloading: 100%|██████████|
Dados carregados com sucesso!
Dimensões do DataFrame: (350144, 8)


,ano,sigla_uf,idade,sexo,cor_raca,peso_amostral,condicao_ocupacao,rendimento_trabalho_habitual
0,2018,BA,23,1,4,48.898,None,<NA>
1,2018,BA,21,2,4,48.898,None,<NA>
2,2018,BA,24,2,4,55.716,None,<NA>
3,2018,BA,23,1,4,57.752,None,<NA>
4,2018,BA,21,1,4,85.379,None,<NA>


In [ ]:
df_pnad.head(100)

,ano,sigla_uf,idade,sexo,cor_raca,peso_amostral,condicao_ocupacao,rendimento_trabalho_habitual
0,2018,BA,23,1,4,48.898,None,<NA>
1,2018,BA,21,2,4,48.898,None,<NA>
2,2018,BA,24,2,4,55.716,None,<NA>
3,2018,BA,23,1,4,57.752,None,<NA>
4,2018,BA,21,1,4,85.379,None,<NA>
...,...,...,...,...,...,...,...,...
95,2018,BA,24,2,4,349.126,2,<NA>
96,2018,BA,25,1,4,309.792,2,<NA>
97,2018,BA,22,2,4,327.640,None,<NA>
98,2018,BA,20,1,2,359.319,None,<NA>


## 3. Preparação dos Dados

Esta é a fase mais crítica. Aqui, vamos:
1.  Limpar os dados e criar as variáveis de resultado.
2.  Definir os grupos de tratamento e controle de forma simulada, como seria feito ao cruzar com dados do Censo Escolar.
3.  Criar as variáveis `treat`, `post` e a interação `did_interaction`.

* Limpeza e Criação de Variáveis

In [ ]:
# Copiando para evitar SettingWithCopyWarning
df_prep = df_pnad.copy()

# 1. Criar variável de resultado: Emprego Formal
# VD4002 == 1 significa "Ocupado"
df_prep['emprego_formal'] = np.where(df_prep['condicao_ocupacao'] == 1, 1, 0)

# 2. Criar variável de resultado: Log da Renda
# Removemos valores nulos e rendas iguais a zero para aplicar o log
df_prep = df_prep.dropna(subset=['rendimento_trabalho_habitual'])
df_prep = df_prep[df_prep['rendimento_trabalho_habitual'] > 0]
df_prep['log_renda'] = np.log(df_prep['rendimento_trabalho_habitual'])

# 3. Simulação da Definição dos Grupos de Tratamento e Controle
# Em um projeto real, cruzaríamos com dados do Censo Escolar por UF.
# Aqui, vamos SIMULAR que um grupo de estados (ex: Nordeste) expandiu as ETIs ("tratamento")
# e outro grupo (ex: Sul) não ("controle").
estados_tratamento = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA']
estados_controle = ['PR', 'SC', 'RS']

# Filtrar o dataframe para incluir apenas os estados definidos
df_prep = df_prep[df_prep['sigla_uf'].isin(estados_tratamento + estados_controle)]

# Criar a variável 'treat'
df_prep['treat'] = np.where(df_prep['sigla_uf'].isin(estados_tratamento), 1, 0)

# 4. Criar a variável 'post' (pós-tratamento)
# Definimos o ano de início da política como 2020
df_prep['post'] = np.where(df_prep['ano'] >= 2020, 1, 0)

# 5. Selecionar colunas finais
colunas_finais = ['ano', 'sigla_uf', 'log_renda', 'emprego_formal', 'treat', 'post', 'idade', 'sexo', 'cor_raca', 'peso_amostral']
df_clean = df_prep[colunas_finais].dropna()

print("Dados preparados para análise.")
print(f"Dimensões finais: {df_clean.shape}")
df_clean.head()

Dados preparados para análise.
Dimensões finais: (94665, 10)


,ano,sigla_uf,log_renda,emprego_formal,treat,post,idade,sexo,cor_raca,peso_amostral
302,2018,BA,3.784,0,1,0,20,1,4,114.387
303,2018,BA,3.784,0,1,0,24,1,4,150.915
304,2018,BA,3.689,0,1,0,24,2,1,254.019
305,2018,BA,3.784,0,1,0,24,1,4,189.972
306,2018,BA,3.689,0,1,0,25,1,4,296.271


## 4. Modelagem: Validando a Premissa Crucial

Antes de estimar o modelo, precisamos verificar a premissa mais importante do DiD: as **tendências paralelas**. Ela assume que, na ausência da política, os grupos de tratamento e controle teriam seguido trajetórias paralelas.

Vamos visualizar as tendências da `log_renda` média para ambos os grupos no período **pré-tratamento** (2016-2019).

* Visualização das Tendências Paralelas

In [ ]:
# Calcular a média ponderada da log_renda por ano e grupo
df_trends = df_clean.copy()
df_trends['renda_ponderada'] = df_trends['log_renda'] * df_trends['peso_amostral']

summary_trends = df_trends.groupby(['ano', 'treat']) \
                          .agg(renda_ponderada_sum=('renda_ponderada', 'sum'),
                               peso_sum=('peso_amostral', 'sum')) \
                          .reset_index()

summary_trends['log_renda_media'] = summary_trends['renda_ponderada_sum'] / summary_trends['peso_sum']

# Gráfico de Tendências Paralelas
plot_trends = (
    ggplot(summary_trends, aes(x='ano', y='log_renda_media', color='factor(treat)'))
    + geom_line(size=1.5)
    + geom_vline(xintercept=2019.5, linetype='dashed', color='red', size=1)
    + labs(
        title="Verificação da Premissa de Tendências Paralelas (Período Pré-Tratamento)",
        subtitle="Log da Renda Média Ponderada (2016-2023)",
        x="Ano",
        y="Log da Renda Média",
        color="Grupo"
    )
    + theme_minimal()
    + scale_color_manual(values=['#0072B2', '#D55E00'], labels=['Controle', 'Tratamento'])
)

print(plot_trends)

<ggplot: (640 x 480)>


### Implementando o Modelo de Diferenças em Diferenças

Com evidências visuais de que as tendências são razoavelmente paralelas no período pré-tratamento, podemos prosseguir com a estimação do modelo de regressão.

A equação é:
$$Y_{it} = \beta_0 + \beta_1 \cdot \text{Treat}_i + \beta_2 \cdot \text{Post}_t + \delta \cdot (\text{Treat}_i \times \text{Post}_t) + \gamma \cdot X_{it} + \epsilon_{it}$$

O coeficiente de interesse é **δ (delta)**, associado à variável de interação `treat:post`. Ele captura o efeito causal da política.

**Importante:** Usaremos Regressão Ponderada (WLS) para incorporar os pesos amostrais da PNAD e calcularemos erros-padrão clusterizados por UF para corrigir a correlação intra-grupo.

* Estimação do Modelo DiD


In [ ]:
# Fórmula do modelo DiD com covariáveis
# C() trata as variáveis como categóricas
formula_did = 'log_renda ~ treat * post + idade + C(sexo) + C(cor_raca)'

# Estimação do modelo WLS (Weighted Least Squares)
# cov_type='cluster' e cov_kwds={'groups': df_clean['UF']} são cruciais para erros-padrão corretos
did_model = smf.wls(
    formula=formula_did,
    data=df_clean,
    weights=df_clean['peso_amostral']
).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_clean['sigla_uf']}
)

# Exibindo os resultados
print("--- RESULTADOS DO MODELO DE DIFERENÇAS EM DIFERENÇAS (DiD) ---")
print(did_model.summary())

--- RESULTADOS DO MODELO DE DIFERENÇAS EM DIFERENÇAS (DiD) ---
                            WLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.043
Model:                            WLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     86.25
Date:                Tue, 15 Jul 2025   Prob (F-statistic):           5.58e-09
Time:                        11:53:22   Log-Likelihood:                -75267.
No. Observations:               94665   AIC:                         1.506e+05
Df Residuals:                   94654   BIC:                         1.507e+05
Df Model:                          10                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------

## 5. Avaliação dos Resultados

O coeficiente da interação `treat:post` é o nosso **estimador DiD**.

**Interpretação:**
* **Coeficiente (`coef`):** O valor estimado para `treat:post` (δ) representa a mudança percentual na renda causada pela política. Se for 0.08, por exemplo, indica um aumento de aproximadamente 8% na renda para o grupo tratado.
* **Significância Estatística (`P>|z|`):** Um valor-p baixo (tipicamente < 0.05) sugere que o efeito estimado não é apenas ruído estatístico.
* **Intervalo de Confiança (`[0.025, 0.975]`):** Fornece uma faixa de valores plausíveis para o efeito real. Se o intervalo não inclui o zero, reforça a conclusão de que o efeito é estatisticamente significativo.

## 6. Implantação: Salvando Dados para o Dashboard

A análise está completa. O passo final no notebook é salvar o dataframe limpo e processado. Este arquivo será a fonte de dados para nosso dashboard interativo em Streamlit.

* Salvando o Arquivo Final

In [ ]:
# Salvar o dataframe limpo para ser usado no aplicativo Streamlit
output_filename = 'pnad_did_para_dashboard.csv'
df_clean.to_csv(output_filename, index=False)

print(f"Arquivo '{output_filename}' salvo com sucesso!")

Arquivo 'pnad_did_para_dashboard.csv' salvo com sucesso!


# Construção do Dashboard Profissional com Streamlit
Agora, vamos criar o dashboard. Crie uma pasta no seu computador para o projeto. Dentro dela, crie um arquivo chamado app.py e uma subpasta chamada pages.

Estrutura de Arquivos:

Projeto_did_PNAD/
│
├── app.py                  # Página principal do dashboard
├── pnad_did_para_dashboard.csv  # Os dados que você salvou do Colab
│
└── pages/
    ├── 1_📊_Análise_Descritiva.py
    └── 2_💡_O_Efeito_Causal.py
Arquivo app.py (Página Principal)


In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

# --- Configuração da Página ---
st.set_page_config(
    page_title="Análise Causal | ETI",
    page_icon="🎓",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Carregamento dos Dados ---
@st.cache_data
def load_data():
    return pd.read_csv('pnad_did_para_dashboard.csv')

df = load_data()

# --- Barra Lateral ---
st.sidebar.title("Sobre o Projeto")
st.sidebar.info(
    """
    Este dashboard apresenta uma análise de **inferência causal** sobre o impacto
    das Escolas de Tempo Integral (ETI) na renda de jovens adultos no Brasil.

    A análise utiliza um modelo de **Diferenças em Diferenças (DiD)**
    com microdados da PNAD Contínua (2016-2023).

    **Navegue pelas páginas para explorar a análise.**
    """
)
st.sidebar.markdown("---")
st.sidebar.write("Desenvolvido com base em metodologias de nível PhD.")

# --- Conteúdo da Página Principal ---
st.title("🎓 Impacto Causal da Escola em Tempo Integral")
st.markdown("### Uma avaliação de políticas públicas através da ciência de dados")
st.markdown("---")

st.header("O Desafio: Medir o Verdadeiro Impacto de uma Política")
st.markdown(
    """
    O Brasil tem investido na expansão das **Escolas de Tempo Integral (ETI)** como uma
    estratégia para melhorar a educação e os resultados futuros dos alunos. Mas como saber
    se o programa realmente funciona?

    Uma simples comparação entre ex-alunos de ETI e de escolas de tempo parcial pode ser enganosa.
    Alunos mais motivados ou de famílias com mais recursos podem ter maior probabilidade de se matricular
    em uma ETI, e esses fatores, não a escola em si, poderiam explicar seu sucesso futuro.
    Isso é o que chamamos de **viés de seleção**.

    Para superar esse desafio, usamos uma abordagem quasi-experimental chamada **Diferenças em Diferenças (DiD)**.
    Esta técnica nos permite isolar o **efeito causal** da política, nos aproximando de uma resposta confiável.
    """
)

st.image(
    "https://placehold.co/1200x400/0072B2/FFFFFF?text=Visualiza%C3%A7%C3%A3o+da+Narrativa+Causal",
    caption="A jornada da correlação para a causalidade."
)

st.info("Use o menu à esquerda para navegar pelas etapas da nossa análise.", icon="👈")

In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.201.162.230:8502

  Stopping...
  Stopping...
